In [1]:
import pandas as pd # type: ignore
import datetime
import calendar
import numpy as np # type: ignore
import re
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from sklearn.ensemble import IsolationForest

pd.set_option('future.no_silent_downcasting', True)

In [2]:
super_sheet = pd.ExcelFile(r"/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI/SSKI EKSTERNAL_25 Okt 2024.xlsx")

In [3]:
def prepare_dataframe(sheet_name, header_count):
    if sheet_name in ['11','16']:
        input_df = super_sheet.parse(sheet_name, header=[3, 2 + header_count])
    else:
        input_df = super_sheet.parse(sheet_name, header=[4, 3 + header_count])

    # Clean up and modify the column headers
    input_df.columns = input_df.columns.map(lambda x: str(x[0]).upper() if 'Unnamed' in str(x) else x)

    # Strip spaces in tuple columns
    input_df.columns = input_df.columns.map(lambda x: (x[0], x[1].strip()) if isinstance(x, tuple) and isinstance(x[1], str) else x)

    # Combine the multi-level column headers into a single string
    input_df.columns = input_df.columns.map(lambda x: f'{x[0]}-{x[1].capitalize()}' if isinstance(x, tuple) else x)

    first_column = input_df.iloc[:, 0]
    second_column = input_df.iloc[:, 1]

    # Filter based on the year (assuming year_to_check is a variable in scope)
    input_df = input_df.filter(regex=str(20))

    # Insert the 'NO' and 'Komponen' columns
    input_df.insert(0, 'NO', first_column)
    input_df.insert(1, 'Komponen', second_column)

    # Remove rows after the 'Keterangan' row (if it exists)
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]

    return input_df

df = prepare_dataframe("4", 2)
df

,NO,Komponen,2005,2006,2007,2008,2009,2010,2011,2012,...,2022-Q2,2022-Q3,2022-Q4,2023,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2
0,1,Core FSIs for Deposit takers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.,.
1,2,Regulatory capital to risk-weighted assets,2.018410e+01,2.125834e+01,1.952652e+01,1.663261e+01,1.715633e+01,1.583715e+01,1.524985e+01,1.616445e+01,...,2.310304e+01,2.352603e+01,2.413405e+01,2.584111e+01,2.321433e+01,2.525690e+01,2.547301e+01,2.584111e+01,24.2906,24.437104
2,3,Total regulatory capital,1.499328e+08,1.680040e+08,1.955812e+08,2.124505e+08,2.404399e+08,2.830622e+08,3.577378e+08,4.380935e+08,...,1.426490e+09,1.474245e+09,1.545912e+09,1.663298e+09,1.507131e+09,1.553433e+09,1.598943e+09,1.663298e+09,1604557738.256399,1651639225.5341
3,4,Risk-weighted assets,7.428265e+08,7.902968e+08,1.001618e+09,1.277313e+09,1.401464e+09,1.787330e+09,2.345845e+09,2.710228e+09,...,6.174467e+09,6.266440e+09,6.405521e+09,6.436634e+09,6.492244e+09,6.150531e+09,6.277008e+09,6.436634e+09,6605673526.403298,6758735617.138998
4,5,Tier 1 capital to risk-weighted assets,1.544958e+01,1.644884e+01,1.558629e+01,1.373710e+01,1.432014e+01,1.400932e+01,1.350456e+01,1.422485e+01,...,2.139775e+01,2.184782e+01,2.255920e+01,2.418378e+01,2.168986e+01,2.365053e+01,2.388114e+01,2.418378e+01,22.744696,22.931249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,212,Total gross loans,8.221774e+08,9.173799e+08,1.107123e+09,1.473261e+09,1.662246e+09,1.961254e+09,2.381914e+09,3.020937e+09,...,6.776339e+09,7.051318e+09,7.295716e+09,7.925380e+09,7.303767e+09,7.543825e+09,7.733890e+09,7.925380e+09,8000771521.443933,8155013645.16169
212,213,Commercial real estate loans to total gross loans,8.656033e-01,1.208650e+00,1.425522e+00,1.525244e+00,1.318092e+00,5.025799e+00,1.483590e+01,1.018863e+01,...,1.544998e+01,1.517310e+01,1.502278e+01,1.469755e+01,1.514551e+01,1.491263e+01,1.476139e+01,1.469755e+01,14.560103,14.999295
213,214,Commercial real estate loans,7.116795e+06,1.108791e+07,1.578228e+07,2.247082e+07,2.190993e+07,9.856868e+07,3.533785e+08,3.077921e+08,...,1.046943e+09,1.069903e+09,1.096020e+09,1.164837e+09,1.106193e+09,1.124982e+09,1.141630e+09,1.164837e+09,1164920582.555511,1223194550.859583
214,215,Total gross loans,8.221774e+08,9.173799e+08,1.107123e+09,1.473261e+09,1.662246e+09,1.961254e+09,2.381914e+09,3.020937e+09,...,6.776339e+09,7.051318e+09,7.295716e+09,7.925380e+09,7.303767e+09,7.543825e+09,7.733890e+09,7.925380e+09,8000771521.443933,8155013645.16169


In [4]:
def prepare_dataframe5(sheet_name):
    if sheet_name == '5a':
        header_array = [3, 4, 5]
    else:
        header_array = [2, 3, 4, 5]

    input_df = super_sheet.parse(sheet_name, header=header_array)

    input_df.columns = input_df.columns.map(lambda x: str(x[0]).upper() if 'Unnamed' in str(x) else x)

    input_df.columns = input_df.columns.map(
        lambda x: tuple(
            str(x[i]).upper() if i == 0 and 'Unnamed' in str(x[1]) else x[i].strip() if isinstance(x[i], str) else x[i]
            for i in range(len(x))
        ) if isinstance(x, tuple) else x
    )

    # Remove row with "keterangan" and rows following
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]
    
    # Gabungkan elemen kolom multi-level menjadi satu string secara fleksibel berdasarkan panjang tuple
    input_df.columns = input_df.columns.map(lambda x: '-'.join([str(i).capitalize() if isinstance(i, str) else str(i) for i in x]) if isinstance(x, tuple) else x)

    first_column = input_df.iloc[:, 0]
    second_column = input_df.iloc[:, 1]

    # Filter columns where the name contains a year (assuming year_to_check is a variable in scope)
    input_df = input_df.filter(regex=str(20))

    # Insert the 'NO' and 'Komponen' columns
    input_df.insert(0, 'NO', first_column)
    input_df.insert(1, 'Komponen', second_column)

    # Remove rows after the 'Keterangan' row (if it exists)
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]

    # Drop columns that contain only a year as their name (e.g., '2020', '2021')
    input_df = input_df.loc[:, ~input_df.columns.str.match(r'^\d{4}$')]

    display(input_df)

    return input_df

In [5]:
def split5a(df):
    claims_df = df.filter(like='Claims')
    liabilities_df = df.filter(like='Liabilities')
    common_columns = df.filter(regex='NO|Komponen')
    # Menggabungkan kolom Nomor dan Komponen ke DataFrame Claims dan Liabilities
    claims_df = pd.concat([common_columns, claims_df], axis=1)
    liabilities_df = pd.concat([common_columns, liabilities_df], axis=1)
    return claims_df, liabilities_df

def split5b(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...)
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_ld = pd.concat([common_columns, df.filter(like='Claims-Loan & deposits')], axis=1)
    liabilities_df_ld = pd.concat([common_columns, df.filter(like='Liabilities-Loan & deposits')], axis=1)
    claims_df_ds = pd.concat([common_columns, df.filter(like='Claims-Debt securities')], axis=1)
    liabilities_df_ds = pd.concat([common_columns, df.filter(like='Liabilities-Debt securities')], axis=1)
    claims_df_oi = pd.concat([common_columns, df.filter(like='Claims-Other instruments')], axis=1)
    liabilities_df_oi = pd.concat([common_columns, df.filter(like='Liabilities-Other instruments')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_ld, liabilities_df_ld, claims_df_ds, liabilities_df_ds, claims_df_oi, liabilities_df_oi

def split5d(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_b = pd.concat([common_columns, df.filter(like='Claims-Banks')], axis=1)
    liabilities_df_b = pd.concat([common_columns, df.filter(like='Liabilities-Banks')], axis=1)
    claims_df_nb = pd.concat([common_columns, df.filter(like='Claims-Nonbanks')], axis=1)
    liabilities_df_nb = pd.concat([common_columns, df.filter(like='Liabilities-Nonbanks')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_b, liabilities_df_b, claims_df_nb, liabilities_df_nb

def split5d1(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_r = pd.concat([common_columns, df.filter(like='Claims-Related offices')], axis=1)
    liabilities_df_r = pd.concat([common_columns, df.filter(like='Liabilities-Related offices')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r

def split5d2(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_nb = pd.concat([common_columns, df.filter(like='Claims-Non-bank financial ins.')], axis=1)
    liabilities_df_nb = pd.concat([common_columns, df.filter(like='Liabilities-Non-bank financial ins.')], axis=1)
    claims_df_nf = pd.concat([common_columns, df.filter(like='Claims-Non-financial corporations')], axis=1)
    liabilities_df_nf = pd.concat([common_columns, df.filter(like='Liabilities-Non-financial corporations')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf

In [6]:
def prepare_all(name, input_df):

    # Step 2: Filter the DataFrame based on the selected columns
    filtered_df = input_df

    # Step 3: Replace '-' with NaN and drop rows with NaN
    filtered_df.replace('-', np.nan, inplace=True)

    # Step 4: Transpose the table, so NO and Komponen become the headers
    transposed_df = filtered_df.set_index(['NO', 'Komponen']).T
    
    return transposed_df

In [7]:
import numpy as np
import pandas as pd

# Month mapping dictionary
month_mapping = {
    'Jan': 'Jan',
    'Feb': 'Feb',
    'Mar': 'Mar',
    'Apr': 'Apr',
    'Mei': 'May',
    'Jun': 'Jun',
    'Jul': 'Jul',
    'Agt': 'Aug',
    'Sept': 'Sep',
    'Okt': 'Oct',
    'Nov': 'Nov',
    'Des': 'Dec',
    'Q1': 'Mar',
    'Q2': 'Jun',
    'Q3': 'Sep',
    'Q4': 'Dec'
}

# Function to reformat Periode
def reformat_periode(periode):
    tokens = periode.split('-')
    
    if len(tokens) > 1:
        year = tokens[0]
        month = tokens[1]
        # Clean the month in case there are additional characters like 'Dec*' or 'OctX'
        month = re.sub(r'[^A-Za-z0-9]', '', month)
        
        # Map the cleaned month to the dictionary
        return f"{year}-{month_mapping.get(month, month)}"
    
    elif len(tokens) == 1:
        # Single-token period handling
        cleaned_token = re.sub(r'[^A-Za-z0-9]', '', tokens[0])  # Clean the single token
        return month_mapping.get(cleaned_token, cleaned_token)
    
    return periode  # Handle unexpected cases gracefully

def detect_outliers_iqr(name,df):
    # Pastikan DataFrame tidak kosong sebelum melanjutkan
    if df.empty:
        print("DataFrame is empty.")
        return pd.DataFrame(columns=['Komponen','Detail', 'Tahun', 'Nilai', 'Batas Bawah', 'Batas Atas', 'Outlier', 'Gap', 'last_period'])
    
    if name == "5.d.2":
        name = "5d2"
    
    # Replace string values (including '-') with NaN
    df = df.replace('-', np.nan).map(lambda x: np.nan if isinstance(x, str) else x)
    
    # Get the last period for each component (assuming index represents period/year)
    last_period = df.index[-1] if len(df.index) > 0 else None

    outliers_data = []
    for column in df.columns:
        # Ensure all values in the column are numeric
        df[column] = pd.to_numeric(df[column], errors='coerce')
        col_data = df[column].dropna()
        
        if col_data.empty:
            continue  # Skip columns with no numeric data
        
        # Calculate IQR, lower bound, and upper bound
        Q1 = col_data.quantile(0.25)
        Q3 = col_data.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Process each value and identify if it's an outlier
        for index, value in df[column].items():
            is_outlier = value < lower_bound or value > upper_bound
            
            # Calculate gap only for outliers, leave empty for non-outliers
            gap = None
            if is_outlier:
                gap = (value - upper_bound) if value > upper_bound else (lower_bound - value)
            token = index.split("-",2)
            if len(token) == 3:
                detail = token[2]
            else:
                detail = '-'


            outliers_data.append({
                'Tabel' : name,
                'No_Komponen':int(column[0]),
                'Komponen': column[1],
                'Detail' : detail,
                'Periode': index,
                'Nilai': value,
                'Batas Bawah': lower_bound,
                'Batas Atas': upper_bound,
                'Outlier': 'Ya' if is_outlier else 'Tidak',
                'Gap': gap,
                'last_period': last_period
            })
    
    # Create the DataFrame with the collected outlier data
    df_outliers = pd.DataFrame(outliers_data)

    # # Apply the function to the Periode column
    df_outliers['Periode'] = df_outliers['Periode'].apply(reformat_periode)
    df_outliers['last_period'] = df_outliers['last_period'].apply(reformat_periode)

    
    return df_outliers

In [8]:
# name = "1"
# raw_df = prepare_dataframe(name, 2)
# checked_df = prepare_all(name, raw_df)
# checked_df = checked_df.iloc[1:]
# outliers_df = detect_outliers_iqr(name,checked_df)
# # Add a new column 'No.' that contains the index values
# outliers_df.insert(0, 'No.', range(1, len(outliers_df) + 1))
# # Display the updated DataFrame
# display(outliers_df)

# output_path = r"/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/application/data/data_pencilan_sski.csv"
# outliers_df.to_csv(output_path, index=False)

In [9]:
import pandas as pd

sheet_names = ["1","2","3","4","5a","5b","5c","5d","5d.1","5.d.2","6","7","8","9","10","11","11a","12","13","14","15","16","16a","17","18","19","20"]  

all_outliers = []

# Loop to process all sheets
for name in sheet_names:
    print(f"Processing SSKI {name}...")

    if name == "5a":
        raw_df = prepare_dataframe5(name)
        
        claims_df, liabilities_df = split5a(raw_df)
        
        claims_df = prepare_all(name, claims_df)
        liabilities_df = prepare_all(name, liabilities_df)
        
        claims_outliers = detect_outliers_iqr(name, claims_df)
        all_outliers.append(claims_outliers)

        liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
        all_outliers.append(liabilities_outliers)

    elif name in ["5b", "5c"]:
        raw_df = prepare_dataframe5(name)
        
        splits = split5b(raw_df)
        
        for claims_df, liabilities_df in zip(splits[::2], splits[1::2]):
            claims_df = prepare_all(name, claims_df)
            claims_outliers = detect_outliers_iqr(name, claims_df)
            all_outliers.append(claims_outliers)

            liabilities_df = prepare_all(name, liabilities_df)
            liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
            all_outliers.append(liabilities_outliers)

    elif name == "5d":
        raw_df = prepare_dataframe5(name)
        
        splits = split5d(raw_df)
        
        for claims_df, liabilities_df in zip(splits[::2], splits[1::2]):
            claims_df = prepare_all(name, claims_df)
            claims_outliers = detect_outliers_iqr(name, claims_df)
            all_outliers.append(claims_outliers)

            liabilities_df = prepare_all(name, liabilities_df)
            liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
            all_outliers.append(liabilities_outliers)

    elif name == "5d.1":
        raw_df = prepare_dataframe5(name)
        
        claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r = split5d1(raw_df)
        
        for df in [claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r]:
            df = prepare_all(name, df)
            outliers = detect_outliers_iqr(name, df)
            all_outliers.append(outliers)

    elif name == "5.d.2":
        raw_df = prepare_dataframe5(name)
        
        claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf = split5d2(raw_df)
        
        for df in [claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf]:
            df = prepare_all(name, df)
            outliers = detect_outliers_iqr(name, df)
            all_outliers.append(outliers)

    else:
        raw_df = prepare_dataframe(name, 2)
        checked_df = prepare_all(name, raw_df)
        checked_df = checked_df.iloc[1:]
        outliers_df = detect_outliers_iqr(name, checked_df)
        all_outliers.append(outliers_df)

# Concatenate all outliers and export to a single CSV file
combined_outliers = pd.concat(all_outliers, ignore_index=True)
combined_outliers.insert(0, 'No.', range(1, len(combined_outliers) + 1))

combined_outliers.to_csv('/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/data/data_pencilan_sski.csv', index=False, sep=";")

Processing SSKI 1...
Processing SSKI 2...
Processing SSKI 3...
Processing SSKI 4...
Processing SSKI 5a...


,NO,Komponen,2021-Jan-Claims,2021-Jan-Liabilities,2021-Feb-Claims,2021-Feb-Liabilities,2021-Mar-Claims,2021-Mar-Liabilities,2021-Apr-Claims,2021-Apr-Liabilities,...,2024-Apr*-Claims,2024-Apr*-Liabilities,2024-May*-Claims,2024-May*-Liabilities,2024-Jun*-Claims,2024-Jun*-Liabilities,2024-Jul*-Claims,2024-Jul*-Liabilities,2024-Aug*-Claims,2024-Aug*-Liabilities
0,1,Seluruh Negara,618232.564,587306.881188,620782.601,588819.779617,633599.116,601129.978079,617088.816,587942.966528,...,701817.376,671829.389,705380.343,675325.638,712180.505,681494.931,706051.636,669756.049,753201.076,713713.377
1,2,Luar Negeri (Nonresiden),19461.197,53195.300000,19826.022,53574.933000,19991.969,53236.475000,19135.509,52794.905000,...,25456.610,53894.175,25492.452,53685.539,26415.385,53584.498,23483.036,53299.959,25421.535,55725.217
2,3,Bank,18497.409,43573.681000,18858.669,43579.729000,19080.439,42691.814000,18225.307,42009.547000,...,22009.787,39790.408,22270.972,39744.795,23053.625,39712.379,20586.271,39493.397,22375.887,41481.090
3,4,a.l. : Antar-Kantor,7700.037,35501.243000,7683.429,35158.382000,7914.313,34437.513000,8115.041,33693.500000,...,10969.200,0.000,22270.972,39744.795,10792.209,0.000,10255.969,0.000,11486.698,0.000
4,5,Non-Bank,963.429,9621.467000,966.931,9995.171000,911.084,10544.583000,909.742,10785.239000,...,3432.894,14068.129,3211.212,13912.131,3348.558,13842.909,2883.963,13766.802,3039.536,14202.995
5,6,Institusi Keuangan Non-Bank,57.402,6034.124000,38.833,6051.701000,36.561,6493.514000,37.505,6755.124000,...,97.056,9604.488,93.286,9521.143,98.922,9493.152,108.256,9339.322,105.288,9904.956
6,7,Sektor Non-Finansial,906.027,3587.343000,928.098,3943.470000,874.523,4051.069000,872.237,4030.115000,...,3335.838,4463.641,3117.926,4390.988,3249.636,4349.757,2775.707,4427.480,2934.248,4298.039
7,8,Indonesia (Residen),598769.151,533968.885188,600954.244,535103.597617,613598.478,547755.588079,597950.914,535009.031528,...,676319.112,617935.214,679844.851,621640.099,685721.736,627910.433,682523.411,616456.090,727734.873,657988.160
8,9,Bank,116524.503,45439.360188,118826.654,46609.874617,124764.632,45332.962079,117139.287,43942.594528,...,122711.345,57891.419,124869.254,58788.878,124156.431,62851.501,123324.936,62407.832,137835.950,72545.907
9,10,a.l. : Antar-Kantor,1838.092,28914.070000,1668.963,28899.005000,1935.600,28152.604000,1770.083,27003.322000,...,1915.874,0.000,1779.226,0.000,1746.096,0.000,1699.019,0.000,1860.781,0.000


Processing SSKI 5b...


,NO,Komponen,2021-Jan-Claims-Total,2021-Jan-Claims-Loan & deposits,2021-Jan-Claims-Debt securities,2021-Jan-Claims-Other instruments,2021-Jan-Liabilities-Total,2021-Jan-Liabilities-Loan & deposits,2021-Jan-Liabilities-Debt securities,2021-Jan-Liabilities-Other instruments,...,2024-Jul*-Liabilities-Debt securities,2024-Jul*-Liabilities-Other instruments,2024-Aug*-Claims-Total,2024-Aug*-Claims-Loan & deposits,2024-Aug*-Claims-Debt securities,2024-Aug*-Claims-Other instruments,2024-Aug*-Liabilities-Total,2024-Aug*-Liabilities-Loan & deposits,2024-Aug*-Liabilities-Debt securities,2024-Aug*-Liabilities-Other instruments
0,1,Seluruh Negara,618232.564,485837.419,113204.582,19190.563,587306.881188,506852.892,12840.062188,67613.927,...,9597.564,82294.463,753201.076,563130.693,164233.439,25836.944,713713.377,614751.009,9662.674,89299.694
1,2,Luar Negeri (Nonresiden),19461.197,12479.162,1597.279,5384.756,53195.300000,27988.176,4026.525000,21180.599,...,3008.399,26606.225,25421.535,13615.836,4502.996,7302.703,55725.217,24586.081,2994.978,28144.158
2,3,Bank,18497.409,12476.610,779.425,5241.374,43573.681000,24931.555,2109.115000,16533.011,...,1523.769,18795.815,22375.887,13615.792,1561.155,7198.940,41481.09,20193.417,1493.553,19794.12
3,4,a.l. : Antar-Kantor,7700.037,2482.034,167.734,5050.269,35501.243000,18944.177,243.262000,16313.804,...,-,-,11486.698,4180.013,379.206,6927.479,-,-,-,-
4,5,Non-Bank,963.429,2.552,817.854,143.023,9621.467000,3056.621,1917.410000,4647.436,...,1484.63,7810.015,3039.536,0.044,2941.841,97.651,14202.995,4351.935,1501.425,8349.635
5,6,Institusi Keuangan Non-Bank,57.402,0.000,0.000,57.402,6034.124000,1242.454,1782.855000,3008.815,...,1338.762,6787.998,105.288,0.044,18.661,86.583,9904.956,1223.646,1347.863,7333.447
6,7,Sektor Non-Finansial,906.027,2.552,817.854,85.621,3587.343000,1814.167,134.555000,1638.621,...,145.868,1022.017,2934.248,0.000,2923.180,11.068,4298.039,3128.289,153.562,1016.188
7,8,Indonesia (Residen),598769.151,473356.041,111607.303,13805.807,533968.885188,478864.608,8813.537188,46290.740,...,6589.165,55688.238,727734.873,549470.189,159730.443,18534.241,657988.16,590164.928,6667.696,61155.536
8,9,Bank,116524.503,96097.639,16348.587,4078.277,45439.360188,11792.021,4043.989188,29603.350,...,3077.476,35744.589,137835.950,73322.312,58743.771,5769.867,72545.907,29488.279,3132.597,39925.031
9,10,a.l. : Antar-Kantor,1838.092,507.157,99.157,1231.778,28914.070000,333.353,49.969000,28530.748,...,-,-,1860.781,433.608,175.441,1251.732,-,-,-,-


Processing SSKI 5c...


...

Processing SSKI 5d.1...


,NO,Komponen,2021-Jan-Claims-Total,2021-Jan-Claims-Related offices,2021-Jan-Liabilities-Total,2021-Jan-Liabilities-Related offices,2021-Feb-Claims-Total,2021-Feb-Claims-Related offices,2021-Feb-Liabilities-Total,2021-Feb-Liabilities-Related offices,...,2024-Jun*-Liabilities-Total,2024-Jun*-Liabilities-Related offices,2024-Jul*-Claims-Total,2024-Jul*-Claims-Related offices,2024-Jul*-Liabilities-Total,2024-Jul*-Liabilities-Related offices,2024-Aug*-Claims-Total,2024-Aug*-Claims-Related offices,2024-Aug*-Liabilities-Total,2024-Aug*-Liabilities-Related offices
0,1,Seluruh Negara Mitra,18497.409,7700.037,43573.681,35501.243,18858.669,7683.429,43579.729,35158.382,...,39712.379,-,20586.271,10255.969,39493.397,-,22375.887,11486.698,41481.090,-
1,2,Amerika,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Amerika Serikat,7914.221,3101.916,3717.588,3320.339,8184.257,2966.084,3930.520,3310.588,...,3413.054,-,8480.396,4472.122,3397.853,-,8310.058,4733.432,3458.825,-
3,4,Kanada,129.996,6.276,5.473,0.001,28.183,6.077,5.884,0.049,...,5.019,-,31.621,0.142,4.296,-,63.337,0.060,0.809,-
4,5,Brazil,0.277,0.000,0.457,0.000,0.153,0.000,0.265,0.000,...,0.258,-,0.000,0.000,0.128,-,0.000,0.000,0.283,-
5,6,Eropa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Belanda,113.642,0.000,26.280,2.001,114.751,0.000,26.963,2.040,...,130.621,-,3.807,0.000,122.527,-,11.851,0.000,123.570,-
7,8,Belgia,71.219,0.000,172.636,0.000,75.356,3.107,163.947,0.000,...,202.778,-,29.344,0.000,74.553,-,65.396,0.000,96.637,-
8,9,Inggris,997.194,533.875,2613.942,2256.825,990.780,514.820,2590.332,2251.280,...,1582.268,-,1118.903,336.284,1623.082,-,1322.965,360.187,1852.515,-
9,10,Jerman,432.389,75.068,665.307,618.239,392.677,58.834,661.108,604.070,...,765.580,-,568.207,87.879,779.664,-,1170.810,242.909,836.528,-


Processing SSKI 5.d.2...


,NO,Komponen,2021-Jan-Claims-Total,2021-Jan-Claims-Non-bank financial ins.,2021-Jan-Claims-Non-financial corporations,2021-Jan-Claims-Others,2021-Jan-Liabilities-Total,2021-Jan-Liabilities-Non-bank financial ins.,2021-Jan-Liabilities-Non-financial corporations,2021-Jan-Liabilities-Others,...,2024-Jul*-Liabilities-Non-financial corporations,2024-Jul*-Liabilities-Others,2024-Aug*-Claims-Total,2024-Aug*-Claims-Non-bank financial ins.,2024-Aug*-Claims-Non-financial corporations,2024-Aug*-Claims-Others,2024-Aug*-Liabilities-Total,2024-Aug*-Liabilities-Non-bank financial ins.,2024-Aug*-Liabilities-Non-financial corporations,2024-Aug*-Liabilities-Others
0,1,Seluruh Negara Mitra,963.429,57.402,793.013,113.014,9621.467,6034.124,2568.272,1019.071,...,2780.940,1646.540,3039.536,105.288,1045.070,1889.178,14202.995,9904.956,2598.849,1699.190
1,2,Amerika,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Amerika Serikat,118.094,6.493,11.956,99.645,550.199,390.597,87.272,72.330,...,75.256,56.116,1822.332,1.249,0.495,1820.588,928.252,815.372,56.102,56.778
3,4,Kanada,0.004,0.000,0.000,0.004,28.917,0.713,12.182,16.022,...,16.545,15.275,0.021,0.000,0.000,0.021,32.967,12.110,4.404,16.453
4,5,Brazil,0.000,0.000,0.000,0.000,4.381,0.000,0.019,4.362,...,0.042,5.379,0.004,0.000,0.000,0.004,5.897,0.000,0.203,5.694
5,6,Eropa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Belanda,0.001,0.000,0.000,0.001,44.530,8.427,10.566,25.537,...,62.511,36.315,0.004,0.000,0.000,0.004,122.964,16.017,68.587,38.360
7,8,Belgia,0.047,0.000,0.047,0.000,9.510,0.578,0.318,8.614,...,0.132,11.363,0.040,0.000,0.000,0.040,12.610,1.407,0.044,11.159
8,9,Inggris,2.081,1.970,0.103,0.008,647.347,581.752,18.739,46.856,...,23.043,45.225,6.494,6.451,0.000,0.043,734.029,657.751,12.996,63.282
9,10,Jerman,1.071,0.000,1.070,0.001,39.278,15.561,3.668,20.049,...,4.113,29.238,0.002,0.000,0.000,0.002,66.400,33.651,3.783,28.966


Processing SSKI 6...
Processing SSKI 7...
Processing SSKI 8...
Processing SSKI 9...
Processing SSKI 10...
Processing SSKI 11...
Processing SSKI 11a...
Processing SSKI 12...
Processing SSKI 13...
Processing SSKI 14...
Processing SSKI 15...
Processing SSKI 16...
Processing SSKI 16a...
Processing SSKI 17...
Processing SSKI 18...
Processing SSKI 19...
Processing SSKI 20...
